In [1]:
import bernn
from bernn import TrainAEThenClassifierHoldout
from ax.service.managed_loop import optimize


2025-05-13 20:47:07.106225: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-13 20:47:07.899044: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2025-05-13 20:47:15.898112: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-13 20:47:15.900975: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [2]:
bernn.__version__

'0.1.7'

In [ ]:
# Minimal example of using Ax to optimize the hyperparameters of a model
class Args:
    def __init__(self):
        self.csv_file = 'adenocarcinoma_data.csv'
        self.path = '../data'
        self.dloss = 'inverseTriplet'
        self.n_epochs = 10

args = Args()


In [ ]:
train = TrainAEThenClassifierHoldout(args, args.path, fix_thres=-1, load_tb=False, 
                                        log_metrics=1, keep_models=False,
                                        log_inputs=False, log_plots=1,
                                        log_tb=False, log_neptune=True, log_mlflow=True, 
                                        groupkfold=1, pools=True)

# train.train()
# List of hyperparameters getting optimized
parameters = [
    {"name": "nu", "type": "range", "bounds": [1e-4, 1e2], "log_scale": False},
    {"name": "lr", "type": "range", "bounds": [1e-4, 1e-2], "log_scale": True},
    {"name": "wd", "type": "range", "bounds": [1e-8, 1e-5], "log_scale": True},
    {"name": "smoothing", "type": "range", "bounds": [0., 0.2]},
    {"name": "margin", "type": "range", "bounds": [0., 10.]},
    {"name": "warmup", "type": "range", "bounds": [1, 10]},
    {"name": "disc_b_warmup", "type": "range", "bounds": [1, 2]},
    {"name": "dropout", "type": "range", "bounds": [0.0, 0.5]},
    {"name": "scaler", "type": "choice",
        "values": ['standard_per_batch', 'standard', 'robust', 'robust_per_batch']},  # scaler whould be no for zinb
    {"name": "layer2", "type": "range", "bounds": [32, 512]},
    {"name": "layer1", "type": "range", "bounds": [512, 1024]},
    
]

# Some hyperparameters are not always required. They are set to a default value in Train.train()
if args.dloss in ['revTriplet', 'revDANN', 'DANN', 'inverseTriplet', 'normae']:
    # gamma = 0 will ensure DANN is not learned
    parameters += [{"name": "gamma", "type": "range", "bounds": [1e-0, 1e2], "log_scale": True}]
if args.variational:
    # beta = 0 because useless outside a variational autoencoder
    parameters += [{"name": "beta", "type": "range", "bounds": [1e-2, 1e2], "log_scale": True}]
if args.zinb:
    # zeta = 0 because useless outside a zinb autoencoder
    parameters += [{"name": "zeta", "type": "range", "bounds": [1e-2, 1e2], "log_scale": True}]
if args.kan and args.use_l1:
    # zeta = 0 because useless outside a zinb autoencoder
    parameters += [{"name": "reg_entropy", "type": "range", "bounds": [1e-4, 1e-2], "log_scale": True}]
if args.use_l1:
    parameters += [{"name": "l1", "type": "range", "bounds": [1e-4, 1e-2], "log_scale": True}]
if args.prune_network:
    parameters += [{"name": "prune_threshold", "type": "range", "bounds": [1e-3, 3e-3], "log_scale": True}]

best_parameters, values, experiment, model = optimize(
    parameters=parameters,
    evaluation_function=train.train,
    objective_name='mcc',
    minimize=False,
    total_trials=args.n_trials,
    random_seed=41,

)


[INFO 05-13 21:01:57] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter nu. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 05-13 21:01:57] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter lr. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 05-13 21:01:57] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter wd. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 05-13 21:01:57] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter smoothing. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO

{'nu': 57.62209361696243, 'lr': 0.0002866874793641167, 'wd': 2.1707730937174506e-06, 'smoothing': 0, 'margin': 9.637901186943054, 'warmup': 363, 'disc_b_warmup': 1, 'dropout': 0.15570732951164246, 'layer2': 227, 'layer1': 956, 'gamma': 1.1582898620708149, 'reg_entropy': 0.005716725144156548, 'l1': 0.00115086353941168, 'prune_threshold': 0.001444340628738408, 'scaler': 'robust_per_batch', 'beta': 0, 'zeta': 0, 'thres': 0}
See results using: tensorboard --logdir=logs/ae_then_classifier_holdout/4dc2c0e9-c467-469d-8cc3-10a902e3fff4 --port=6006
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/ADLab/BERNN/e/BE-434
Rep: 0
[neptune] [error  ] Error occurred during asynchronous operation processing: 

----NeptuneEmptyLocationException----------------------------------------------

Neptune could not find files in the requested location (../data/subjects_experiment_ATN_verified_diagnosis.csv) during the creation of an Artifact in "metadata".

Need help?-> https://d